In [1]:
from app.aux.entropyZero import int_cp_T
from app.aux.read_data import ReadData
import pyomo.environ as pyo
import numpy as np
import pandas as pd

In [2]:
data_path = 'Informations.csv'
data, species, initial, components = ReadData(data_path).get_infos()
total_species = len(species)

In [3]:
def identify_phases(phase_type, data):
    """
    Identifies the components that belong to the given phase type ('s' for solids, 'g' for gases).
    """
    return [i for i, comp in enumerate(data) if data[comp].get("Phase") == phase_type]

In [4]:
gas_comps = identify_phases('g',data)
solid_comps = identify_phases('s',data)

In [5]:
A = np.array([[component[specie] for specie in species] for component in data.values()])

def bnds_values(initial, data, inhibited_component=None):
    """
    Returns the bounds for the variables based on the initial guess and the inhibited component.
    """
    max_species = np.dot(initial, A)
    epsilon = 1e-05
    bnds_aux = []

    if inhibited_component and inhibited_component != '---':
        try:
            aux_idx = next(index for index, value in data.items() if value['Component'] == inhibited_component)
        except StopIteration:
            print(f"Inhibited component '{inhibited_component}' not found.")
            aux_idx = None
    else:
        aux_idx = None

    for i, comp in enumerate(data):
        if aux_idx is not None and i == aux_idx:
            bnds_aux.append((1e-8, epsilon))
        else:
            a = np.multiply(1 / np.where(A[i] != 0, A[i], np.inf), max_species)
            upper_bound = np.min(a[a > 0]) if a[a > 0].size > 0 else epsilon
            bnds_aux.append((1e-8, max(upper_bound, epsilon)))

    return tuple(bnds_aux)

In [ ]:
total_components = len(components)

def solve_entropy(initial, Tinit, P):
    initial[initial == 0] = 0.00001
    bnds = bnds_values(initial, data)
    model = pyo.ConcreteModel()
    model.n = pyo.Var(range(total_components), domain=pyo.NonNegativeReals, bounds=lambda m, i: bnds[i])
    model.T = pyo.Var(domain=pyo.NonNegativeReals, bounds=(500, 1600), initialize=Tinit)

    gases = identify_phases('g', data)

    def entropy_rule(model):
        T0 = 298.15
        R = 8.314  # J/mol·K

        int_cp_T_values, deltaH, deltaG = int_cp_T(model.T, data)

        mi_gas = [
            ((deltaH[i] - deltaG[i]) / T0) - R * pyo.log(P) - R * pyo.log(model.n[i] / sum(model.n) + int_cp_T_values[i])
            for i in gases
        ]

        regularization_term = 1e-6
        total_gibbs = sum(mi_gas[i] * model.n[gases[i]] for i in range(len(mi_gas))) + regularization_term
        
        return total_gibbs

    model.obj = pyo.Objective(rule=entropy_rule, sense=pyo.maximize)
    model.element_balance = pyo.ConstraintList()
    for i in range(total_species):
        tolerance = 1e-8
        lhs = sum(A[j, i] * model.n[j] for j in range(total_components))
        rhs = sum(A[j, i] * initial[j] for j in range(total_components))
        model.element_balance.add(pyo.inequality(-tolerance, lhs - rhs, tolerance))

    solver = pyo.SolverFactory('ipopt')
    solver.options['tol'] = 1e-1
    solver.options['max_iter'] = 5000
    solver.options['acceptable_tol'] = 1e-6
    solver.options['acceptable_obj_change_tol'] = 1e-6
    results = solver.solve(model, tee=False)

    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        optimized_n = [pyo.value(model.n[i]) for i in range(total_components)]
        Teq = pyo.value(model.T)
        return optimized_n, Teq
    else:
        raise Exception("Optimal solution not found.")

In [7]:
res = solve_entropy(initial,1000,220)
res

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
data

{0: {'Component': 'Methane',
  'Phase': 'g',
  'a': 1.702,
  'b': 0.009081,
  'c': -2.164e-06,
  'd': 0.0,
  '∆Hf298': -74520.0,
  '∆Gf298': -50460.0,
  'Pc': 45.99,
  'Tc': 190.6,
  'omega': 0.012,
  'Tmax': 1500,
  'initial': 0.0,
  'C': 1,
  'H': 4.0,
  'O': 0.0,
  'N': 0.0},
 1: {'Component': 'Water',
  'Phase': 'g',
  'a': 3.47,
  'b': 0.00145,
  'c': 0.0,
  'd': 12100.0,
  '∆Hf298': -241818.0,
  '∆Gf298': -228572.0,
  'Pc': 220.55,
  'Tc': 647.1,
  'omega': 0.345,
  'Tmax': 1500,
  'initial': 2.0,
  'C': 0,
  'H': 2.0,
  'O': 1.0,
  'N': 0.0},
 2: {'Component': 'CarbonMonoxide',
  'Phase': 'g',
  'a': 3.376,
  'b': 0.000557,
  'c': 0.0,
  'd': -3100.0,
  '∆Hf298': -110525.0,
  '∆Gf298': -137169.0,
  'Pc': 34.99,
  'Tc': 132.9,
  'omega': 0.048,
  'Tmax': 1500,
  'initial': 0.0,
  'C': 1,
  'H': 0.0,
  'O': 1.0,
  'N': 0.0},
 3: {'Component': 'CarbonDioxide',
  'Phase': 'g',
  'a': 5.457,
  'b': 0.001045,
  'c': 0.0,
  'd': -115700.0,
  '∆Hf298': -393509.0,
  '∆Gf298': -394359.0,
